In [45]:
%pip install flask_cors

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: C:\Users\nina\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [71]:
#imports
import pandas as pd
import os
import sqlite3 as sq3
import datetime as date
import json
import uuid
from typing import List
from flask import Flask
from collections import OrderedDict
from flask_cors import CORS, cross_origin

In [47]:
#prepare data and put into correct format for LostIslandBuildings
def prepareData():

    df = pd.read_csv('LostIslandBuildings.csv')

    #Rename Color / Area column to be Color
    df.rename(columns={'Color / Area': 'Color'}, inplace=True)

    #Separate Coordinates into X-Coord and Y-Coord
    df[['X-Coord', 'Y-Coord']] = df['Coordinates'].str.split(' ', expand=True)
    df = df.drop('Coordinates', axis=1)
    df['X-Coord'] = df['X-Coord'].str[2:]
    df['Y-Coord'] = df['Y-Coord'].str[2:]

    #Separate Buff Percent into categories
    buffCategories = ['HP Guardian', 'HP Shooter', 'HP Carrier', 'ATK Guardian', 'ATK Shooter', 'ATK Carrier', 'DEF Guardian', 'DEF Shooter', 'DEF Carrier']

    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df['Buff Percent'].apply(lambda x: x.split(' ')[0] if buffCategories[i] in x else 0)

    #Separate Buff Percent into categories for Wonders
    wonderCondition = df['Alliance Influence'] == '25,000'
    wonderRowsOnly = df.loc[wonderCondition, :]
    wonderRowBuffs = ['3.6%', '3.6%', '3.6%', '9%', '9%', '9%', '9%', '9%', '9%']
    for i in range(len(buffCategories)):
        df.loc[wonderCondition, buffCategories[i]] = wonderRowBuffs[i]
    df.loc[wonderCondition, buffCategories] = wonderRowsOnly[buffCategories]

    df = df.drop('Buff Percent', axis=1)

    #Separate Soldier Production into categories
    productionCategories = ['Guardian', 'Shooter', 'Carrier']

    for i in range(len(productionCategories)):
        df[productionCategories[i] + ' Output'] = df['Soldier Production'].apply(lambda x: x.split(' ')[0] if productionCategories[i] in x else 0)

    #Separate Soldier Production for Wonders
    wonderRowProduction = ['44/h', '44/h', '44/h']
    productionCategoriesRenamed = ['Guardian Output', 'Shooter Output', 'Carrier Output']
    for i in range(len(productionCategoriesRenamed)):
        wonderRowsOnly[productionCategoriesRenamed[i]] = wonderRowProduction[i]
    df.loc[wonderCondition, productionCategoriesRenamed] = wonderRowsOnly[productionCategoriesRenamed]

    df = df.drop('Soldier Production', axis=1)

    #Rename Buildings to Building
    df.rename(columns={'Buildings': 'Building'}, inplace=True)

    #Formatting
    df = df.apply(lambda x: x.str.replace('%', '', regex=True))
    df = df.apply(lambda x: x.str.replace('/h', '', regex=True))
    df = df.apply(lambda x: x.str.replace(',', '', regex=True))
    df = df.fillna(0)

    df['Alliance Influence'] = df['Alliance Influence'].astype('int')
    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df[buffCategories[i]].astype(float)
    for i in range(len(productionCategoriesRenamed)):
        df[productionCategoriesRenamed[i]] = df[productionCategoriesRenamed[i]].astype('int')

    df.to_csv('UpdatedLostIslandBuildings.csv', index=False)

In [48]:
#Convert csv file to SQLite database
def convertToSQLiteDB():
    updatedDf = pd.read_csv('UpdatedLostIslandBuildings.csv')
    conn = sq3.connect('LostIslandBuildings.db')
    updatedDf.to_sql('Buildings', conn, if_exists='replace', index=False)
    conn.close()

In [49]:
#Create empty SQLite DB with only the headers initialized for a new Lost Island and setup textfile with alliance names
def initializeNewIsland(pinkName="", blueName="", brownName="", greenName="", purpleName="", yellowName="", tealName="", orangeName=""):
    current_date = date.datetime.now().strftime("%Y-%m-%d")
    save_path = "Events/" + current_date
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    #Connect to the sq3 db
    conn = sq3.connect(save_path + "/LostIslandAllianceBuildings.db")
    #Create cursor obj to exec SQL commands
    cursor = conn.cursor()

    #Table schema
    table_schema = """
    CREATE TABLE IF NOT EXISTS alliance_buildings (
        id INTEGER PRIMARY KEY,
        Alliance TEXT,
        Building TEXT,
        Color TEXT,
        "Alliance Influence" INTEGER,
        "X-Coord" INTEGER,
        "Y-Coord" INTEGER,
        "HP Guardian" FLOAT,
        "HP Shooter" FLOAT,
        "HP Carrier" FLOAT,
        "ATK Guardian" FLOAT,
        "ATK Shooter" FLOAT,
        "ATK Carrier" FLOAT,
        "DEF Guardian" FLOAT,
        "DEF Shooter" FLOAT,
        "DEF Carrier" FLOAT,
        "Guardian Output" INTEGER,
        "Shooter Output" INTEGER,
        "Carrier Output" INTEGER
    );"""
    cursor.execute(table_schema)
    conn.commit()
    conn.close()

    #Create config file
    islandConfig = save_path + "/config.json"

    islandConfigData = {
        "date": current_date,
        "alliances": {
            "pink": pinkName,
            "blue": blueName,
            "brown": brownName,
            "green": greenName,
            "purple": purpleName,
            "yellow": yellowName,
            "teal": tealName,
            "orange": orangeName
        }
    }

    with open(islandConfig, 'w') as configFile:
        json.dump(islandConfigData, configFile)
        
#testing
#initializeNewIsland("EOA", "", "LAO", "SEE", "SeN", "", "TAO")

In [50]:
#Class for a building occupied by an alliance
class AllianceOccupiedBuilding:
    def __init__(self, alliance, xCoord, yCoord):
        self.alliance = alliance
        self.xCoord = xCoord
        self.yCoord = yCoord
    
    def __str__(self):
        return f"Building occupied by {self.alliance} at Coordinates ({self.xCoord}, {self.yCoord})"
    #Getters
    def getAlliance(self): return self.alliance
    def getXCoord(self): return self.xCoord
    def getYCoord(self): return self.yCoord

In [51]:
#Add Building to Alliance Buildings given X-Y coordinates and the occupying alliance
def addBuilding(date, building: AllianceOccupiedBuilding):
    buildingPath = "LostIslandBuildings.db"
    allianceBuildingPath = "Events/" + date + "/LostIslandAllianceBuildings.db"

    #Fetch building information
    buildingConn = sq3.connect(buildingPath)
    cursor = buildingConn.cursor()
    cursor.execute('SELECT * FROM Buildings WHERE "X-Coord" = ? AND "Y-Coord" = ?', (building.getXCoord(), building.getYCoord())) 
    buildingInfo = cursor.fetchall()
    cursor.close()
    buildingConn.close()

    #Format data
    buildingName = buildingInfo[0][0]
    color = buildingInfo[0][1]
    influence = int(buildingInfo[0][2])
    hpGuard = float(buildingInfo[0][5])
    hpShoot = float(buildingInfo[0][6])
    hpCarr = float(buildingInfo[0][7])
    atkGuard = float(buildingInfo[0][8])
    atkShoot = float(buildingInfo[0][9])
    atkCarr = float(buildingInfo[0][10])
    defGuard = float(buildingInfo[0][11])
    defShoot = float(buildingInfo[0][12])
    defCarr = float(buildingInfo[0][13])
    guardOut = int(buildingInfo[0][14])
    shootOut = int(buildingInfo[0][15])
    carrOut = int(buildingInfo[0][16])

    #Add to alliance building
    allianceBuildingConn = sq3.connect(allianceBuildingPath)
    cursor1 = allianceBuildingConn.cursor()
    #Check if already exists
    cursor1.execute('SELECT Alliance FROM alliance_buildings WHERE "X-Coord" = ? AND "Y-Coord" = ?', (building.getXCoord(), building.getYCoord()))
    existingBuildingRow = cursor1.fetchone()
    if not existingBuildingRow:
        cursor1.execute('INSERT INTO alliance_buildings (Alliance, Building, Color, "Alliance Influence", "X-Coord", "Y-Coord", "HP Guardian", "HP Shooter", "HP Carrier", "ATK Guardian", "ATK Shooter", "ATK Carrier", "DEF Guardian", "DEF Shooter", "DEF Carrier", "Guardian Output", "Shooter Output", "Carrier Output") buffTotalsS (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', (building.getAlliance(), buildingName, color, influence, building.getXCoord(), building.getYCoord(), hpGuard, hpShoot, hpCarr, atkGuard, atkShoot, atkCarr, defGuard, defShoot, defCarr, guardOut, shootOut, carrOut))
    #Case: check if exists under different alliance
    elif existingBuildingRow != building.getAlliance():
         cursor1.execute('UPDATE alliance_buildings SET Alliance = ? WHERE "X-Coord" = ? AND "Y-Coord" = ?', (building.getAlliance(), building.getXCoord(), building.getYCoord()))
    allianceBuildingConn.commit()
    allianceBuildingConn.close()

In [52]:
#calls addBuilding function on a list of AllianceOccupiedBuilding objects
def addGroupOfBuildings(date, allianceBuildingList: List[AllianceOccupiedBuilding]):
    for building in allianceBuildingList:
        addBuilding(date, building)

### Array of JSON Data: Passed in from user input

```json
[
    {
        "alliance": text,
        "xcoord": int,
        "ycoord": int
    },
    ...
]
```

In [53]:
#converts array to a list of AllianceOccupiedBuilding objects
def convertsArraytoListofAllianceOccupiedBuilding(date, arrayData):
    listOfAllianceOccupiedBuildings = []
    for building in arrayData:
        listOfAllianceOccupiedBuildings.append(AllianceOccupiedBuilding(building['alliance'], building['xcoord'], building['ycoord']))
    return listOfAllianceOccupiedBuildings

#testing data
#fake_entries = [
#    {"alliance": "Alliance A", "xcoord": 10, "ycoord": 20},
#    {"alliance": "Alliance B", "xcoord": 30, "ycoord": 40},
#    {"alliance": "Alliance C", "xcoord": 50, "ycoord": 60},
#    {"alliance": "Alliance D", "xcoord": 70, "ycoord": 80},
#    {"alliance": "Alliance E", "xcoord": 90, "ycoord": 100},
#    {"alliance": "Alliance F", "xcoord": 110, "ycoord": 120},
#    {"alliance": "Alliance G", "xcoord": 130, "ycoord": 140},
#    {"alliance": "Alliance H", "xcoord": 150, "ycoord": 160},
#    {"alliance": "Alliance I", "xcoord": 170, "ycoord": 180},
#    {"alliance": "Alliance J", "xcoord": 190, "ycoord": 200}
#]
#result = convertsArraytoListofAllianceOccupiedBuilding("2023-10-21", fake_entries)
#for i in result:
#    print(i)


In [54]:
#delete a specific ID from an Alliance's buildings
def deleteIDFromAllianceBuildings(date, building: AllianceOccupiedBuilding):
    allianceBuildingsPath = 'Events/' + date + '/LostIslandAllianceBuildings.db'
    allianceBuildingsConn = sq3.connect(allianceBuildingsPath)
    cursor = allianceBuildingsConn.cursor()
    cursor.execute('DELETE from alliance_buildings WHERE alliance = ? AND "X-Coord" = ? AND "Y-Coord" = ?', (building.getAlliance(), building.getXCoord(), building.getYCoord()))
    allianceBuildingsConn.commit()
    allianceBuildingsConn.close()

In [55]:
#Get totals for buffs
def getBuffTotalsForAlliance(date, allianceName):
    allianceBuildingsPath = 'Events/' + date + '/LostIslandAllianceBuildings.db'
    allianceBuildingsConn = sq3.connect(allianceBuildingsPath)
    cursor = allianceBuildingsConn.cursor()
    cursor.execute('SELECT SUM("Alliance Influence"), SUM("HP Guardian"), SUM("HP Shooter"), SUM("HP Carrier"), SUM("ATK Guardian"), SUM("ATK Shooter"), SUM("ATK Carrier"), SUM("DEF Guardian"), SUM("DEF Shooter"), SUM("DEF Carrier"), SUM("Guardian Output"), SUM("Shooter Output"), SUM("Carrier Output") from alliance_buildings WHERE Alliance = ?', (allianceName,))
    allSums = cursor.fetchone()
    allianceInfluenceSum = allSums[0]
    hpGuardianSum = allSums[1]
    hpShooterSum = allSums[2]
    hpCarrierSum = allSums[3]
    atkGuardianSum = allSums[4]
    atkShooterSum = allSums[5]
    atkCarrierSum = allSums[6]
    defGuardianSum = allSums[7]
    defShooterSum = allSums[8]
    defCarrierSum = allSums[9]
    guardianOutSum = allSums[10]
    shooterOutSum = allSums[11]
    carrierOutSum = allSums[12]

    outputJson = {
        "key": int(uuid.uuid4()),
        "alliance": allianceName,
        "alliance influence sum": allianceInfluenceSum if allianceInfluenceSum is not None else 0.0,
        "hp guardian sum": hpGuardianSum if hpGuardianSum is not None else 0.0,
        "hp shooter sum": hpShooterSum if hpShooterSum is not None else 0.0,
        "hp carrier sum": hpCarrierSum if hpCarrierSum is not None else 0.0,
        "atk guardian sum": atkGuardianSum if atkGuardianSum is not None else 0.0,
        "atk shooter sum": atkShooterSum if atkShooterSum is not None else 0.0,
        "atk carrier sum": atkCarrierSum if atkCarrierSum is not None else 0.0,
        "def guardian sum": defGuardianSum if defGuardianSum is not None else 0.0,
        "def shooter sum": defShooterSum if defShooterSum is not None else 0.0,
        "def carrier sum": defCarrierSum if defCarrierSum is not None else 0.0,
        "guardian output sum": guardianOutSum if guardianOutSum is not None else 0,
        "shooter output sum": shooterOutSum if shooterOutSum is not None else 0,
        "carrier output sum": carrierOutSum if carrierOutSum is not None else 0
    }

    return outputJson

print(getBuffTotalsForAlliance("2023-10-21", "ARA"))

{'key': 98193837334045682259315142281996126073, 'alliance': 'ARA', 'alliance influence sum': 27000, 'hp guardian sum': 0.0, 'hp shooter sum': 0.0, 'hp carrier sum': 0.0, 'atk guardian sum': 0.0, 'atk shooter sum': 0.0, 'atk carrier sum': 0.0, 'def guardian sum': 10.0, 'def shooter sum': 86.0, 'def carrier sum': 48.0, 'guardian output sum': 22, 'shooter output sum': 0, 'carrier output sum': 17}


In [56]:
#Get totals for buffs for all alliances in single Lost Island
def getAllBuffTotals(date):
    alliancePath = "Events/" + date + "/config.json"
    allAllianceBuffTotal = []
    with open(alliancePath, 'r') as jsonFile:
        configData = json.load(jsonFile)
        for color, alliance in configData['alliances'].items():
            #only for colors that are assigned to an alliance
            if alliance != '':
                allAllianceBuffTotal.append(getBuffTotalsForAlliance(date, alliance))
    return allAllianceBuffTotal
#testing
buffTotals = getAllBuffTotals("2023-10-21")
print(buffTotals)

[{'key': 60127194336806196326101733069113041866, 'alliance': 'ARA', 'alliance influence sum': 27000, 'hp guardian sum': 0.0, 'hp shooter sum': 0.0, 'hp carrier sum': 0.0, 'atk guardian sum': 0.0, 'atk shooter sum': 0.0, 'atk carrier sum': 0.0, 'def guardian sum': 10.0, 'def shooter sum': 86.0, 'def carrier sum': 48.0, 'guardian output sum': 22, 'shooter output sum': 0, 'carrier output sum': 17}, {'key': 226179943279181356784481228218638737291, 'alliance': 'LAO', 'alliance influence sum': 0.0, 'hp guardian sum': 0.0, 'hp shooter sum': 0.0, 'hp carrier sum': 0.0, 'atk guardian sum': 0.0, 'atk shooter sum': 0.0, 'atk carrier sum': 0.0, 'def guardian sum': 0.0, 'def shooter sum': 0.0, 'def carrier sum': 0.0, 'guardian output sum': 0, 'shooter output sum': 0, 'carrier output sum': 0}, {'key': 319307894995496095740140775212901472646, 'alliance': 'SEE', 'alliance influence sum': 0.0, 'hp guardian sum': 0.0, 'hp shooter sum': 0.0, 'hp carrier sum': 0.0, 'atk guardian sum': 0.0, 'atk shooter su

In [57]:
#Updates which alliance is the highest for each buff
def getAllHighestBuffs(date, buffTotals):
    highestBuffsOutput = {
        "alliance influence sum": (0, ""),
        "hp guardian sum": (0.0, ""),
        "hp shooter sum": (0.0, ""),
        "hp carrier sum": (0.0, ""),
        "atk guardian sum": (0.0, ""),
        "atk shooter sum": (0.0, ""),
        "atk carrier sum": (0.0, ""),
        "def guardian sum": (0.0, ""),
        "def shooter sum": (0.0, ""),
        "def carrier sum": (0.0, ""), 
        "guardian output sum": (0, ""),
        "shooter output sum": (0, ""),
        "carrier output sum": (0, "")
    }
    for i in range(len(buffTotals)):
        if buffTotals[0]['alliance influence sum'] > highestBuffsOutput['alliance influence sum'][0]:
            highestBuffsOutput['alliance influence sum'] = (buffTotals[0]['alliance influence sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['hp guardian sum'] > highestBuffsOutput['hp guardian sum'][0]:
            highestBuffsOutput['hp guardian sum'] = (buffTotals[0]['hp guardian sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['hp shooter sum'] > highestBuffsOutput['hp shooter sum'][0]:
            highestBuffsOutput['hp shooter sum'] = (buffTotals[0]['hp shooter sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['hp carrier sum'] > highestBuffsOutput['hp carrier sum'][0]:
            highestBuffsOutput['hp carrier sum'] = (buffTotals[0]['hp carrier sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['atk guardian sum'] > highestBuffsOutput['atk guardian sum'][0]:
            highestBuffsOutput['atk guardian sum'] = (buffTotals[0]['atk guardian sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['atk shooter sum'] > highestBuffsOutput['atk shooter sum'][0]:
            highestBuffsOutput['atk shooter sum'] = (buffTotals[0]['atk shooter sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['atk carrier sum'] > highestBuffsOutput['atk carrier sum'][0]:
            highestBuffsOutput['atk carrier sum'] = (buffTotals[0]['atk carrier sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['def guardian sum'] > highestBuffsOutput['def guardian sum'][0]:
            highestBuffsOutput['def guardian sum'] = (buffTotals[0]['def guardian sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['def shooter sum'] > highestBuffsOutput['def shooter sum'][0]:
            highestBuffsOutput['def shooter sum'] = (buffTotals[0]['def shooter sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['def carrier sum'] > highestBuffsOutput['def carrier sum'][0]:
            highestBuffsOutput['def carrier sum'] = (buffTotals[0]['def carrier sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['guardian output sum'] > highestBuffsOutput['guardian output sum'][0]:
            highestBuffsOutput['guardian output sum'] = (buffTotals[0]['guardian output sum'], buffTotals[0]['alliance'])
        if buffTotals[0]['shooter output sum'] > highestBuffsOutput['shooter output sum'][0]:
            highestBuffsOutput['shooter output sum'] = (buffTotals['shooter output sum'], buffTotals['alliance'])
        if buffTotals[0]['carrier output sum'] > highestBuffsOutput['carrier output sum'][0]:
            highestBuffsOutput['carrier output sum'] = (buffTotals[0]['carrier output sum'], buffTotals[0]['alliance'])
    return highestBuffsOutput
            
#testing
#getAllHighestBuffs("2023-10-21", buffTotals)

In [58]:
#Updates entire output.json file with buff totals and the highest buffs
def updateOutputConfig(date):
    path = "Events/" + date + "/output.json"
    buffTotals = getAllBuffTotals(date)
    highestBuffs = getAllHighestBuffs(date, buffTotals)
    outputJson = {
        "buff totals": buffTotals,
        "highest buffs": highestBuffs
    }
    with open(path, 'w') as jsonFile:
        json.dump(outputJson, jsonFile)
#testing
updateOutputConfig("2023-10-21")

In [72]:
app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "http://localhost:3000"}})

#GET Alliance Buildings for a Specific Alliance
@app.route('/api/getAllianceBuildings/<date>/<alliancename>', methods=['GET'])
@cross_origin()
def getAllianceBuildingsRequest(date, alliancename):
    allianceBuildingsPath = "./Events/" + date + "/LostIslandAllianceBuildings.db"
    allianceBuildingsConn = sq3.connect(allianceBuildingsPath)
    cursor = allianceBuildingsConn.cursor()
    cursor.execute('SELECT id AS "key", Building AS "building", Alliance AS "alliance", "Alliance Influence" AS "influence", "X-Coord" AS "x", "Y-Coord" AS "y", "Guardian Output" AS "guardout", "Shooter Output" AS "shootout", "Carrier Output" AS "carrout", "HP Guardian" AS "hpguard", "HP Shooter" AS "hpshoot", "HP Carrier" AS "hpcarr", "ATK Guardian" AS "atkguard", "ATK Shooter" AS "atkshoot", "ATK Carrier" AS "atkcarr", "DEF Guardian" AS "defguard", "DEF Shooter" AS "defshoot", "DEF Carrier" AS "defcarr" FROM alliance_buildings WHERE Alliance = ? ORDER BY "Alliance Influence" DESC', (alliancename,))
    headerNames = [desc[0] for desc in cursor.description]
    resultRows = cursor.fetchall()
    cursor.close()
    allianceBuildingsConn.close()
    
    #add header rows to the resulting json array
    result=[]
    for row in resultRows:
        ordered_dict = OrderedDict()
        for header, data in zip(headerNames, row):
            ordered_dict[header] = data
        result.append(ordered_dict)
    return result

In [73]:
#GET Alliances in specific Lost Island date
@app.route('/api/getAllAlliances/<date>', methods=['GET'])
@cross_origin()
def getAllAlliancesRequest(date):
    datePath = "./Events/" + date + "/config.json"
    with open(datePath, 'r') as file:
        data = json.load(file)
    #remove colors that aren't associated with alliance
    justAlliances = data["alliances"]
    filteredAlliances = {key: value for key, value in justAlliances.items() if value != ""}
    newData = {
        "date": data["date"],
        "alliances": filteredAlliances
    }
    return newData

In [74]:
#GET Alliance Total
@app.route('/api/getAllianceTotal/<date>/<alliancename>', methods=['GET'])
@cross_origin()
def getAllianceTotalRequest(date, alliancename):
    return getBuffTotalsForAlliance(date, alliancename)

In [75]:
#GET All Alliances Total
@app.route('/api/getAlliancesTotal/<date>', methods=['GET'])
@cross_origin()
def getAlliancesTotalRequest(date):
    return getAllBuffTotals(date)

In [76]:
#to run the flask app
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Oct/2023 09:47:44] "GET /api/getAllAlliances/2023-10-21 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 09:47:44] "GET /api/getAllianceBuildings/2023-10-21/LAO HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 09:47:44] "GET /api/getAllianceTotal/2023-10-21/LAO HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:25] "GET /api/getAllianceBuildings/2023-10-21/SEE HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:25] "GET /api/getAllianceTotal/2023-10-21/SEE HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:26] "GET /api/getAllianceBuildings/2023-10-21/SeN HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:26] "GET /api/getAllianceTotal/2023-10-21/SeN HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:27] "GET /api/getAllianceBuildings/2023-10-21/TAO HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:27] "GET /api/getAllianceTotal/2023-10-21/TAO HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2023 10:27:27] "GET /api/getAllianceBuildings/2023-10-2